<a href="https://colab.research.google.com/github/Otwooo/music_composition_AI/blob/main/No.01%20music_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **작곡 AI 만들기**
악보가 주어졌을 때 다음 음으로는 무엇이 오는지 예측해보자. 한음한음 차례대로 예측하다보면 노래가 완성된다.

### **1. 라이브러리 불러오기 & 데이터 불러오기**

In [6]:
import numpy as np
import tensorflow as tf

In [9]:
text = open('/content/drive/MyDrive/강의/Coding Apple - 문과여도 상관없다 Tensorflow 딥러닝 AI 기초부터 실무까지/pianoabc.txt', 'r').read()
print(text[:100])

e"G"d2B"D"A3/2B/2c"G"B2G"D"A2e"G"d2B"D"A3/2B/2c:2/4"F"BF:3/4"G"G2e:"C"g2e"Bb"f2d"F"c2AF2e"C"g2e"Bb"f


### **2. 데이터 전처리**

#### **2.1 문자 데이터를 숫자로 바꿔주기**
bag of word(단어 주머니)를 만들어 문자 데이터를 숫자 데이터로 바꿔준다.

In [10]:
unique_text = list(set(text))
unique_text.sort()
print(unique_text)

['"', "'", ',', '/', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', 'A', 'B', 'C', 'D', 'E', 'F', 'G', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'z']


(문자 -> 숫자) (숫자 -> 문자) 둘다 만들어 두면 유용하다.

In [11]:
text_to_num = {}
num_to_text = {}

for i, data in enumerate(unique_text):
  text_to_num[data] = i
  num_to_text[i] = data

print(text_to_num)
print(num_to_text)

{'"': 0, "'": 1, ',': 2, '/': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, ':': 13, 'A': 14, 'B': 15, 'C': 16, 'D': 17, 'E': 18, 'F': 19, 'G': 20, '^': 21, '_': 22, 'a': 23, 'b': 24, 'c': 25, 'd': 26, 'e': 27, 'f': 28, 'g': 29, 'z': 30}
{0: '"', 1: "'", 2: ',', 3: '/', 4: '1', 5: '2', 6: '3', 7: '4', 8: '5', 9: '6', 10: '7', 11: '8', 12: '9', 13: ':', 14: 'A', 15: 'B', 16: 'C', 17: 'D', 18: 'E', 19: 'F', 20: 'G', 21: '^', 22: '_', 23: 'a', 24: 'b', 25: 'c', 26: 'd', 27: 'e', 28: 'f', 29: 'g', 30: 'z'}


각 문자에 맞는 숫자와 각 숫자에 맞는 문자로 구성된 딕셔너리 생성

In [12]:
num_text = []

for i in text:
  num_text.append(text_to_num[i])

print(num_text[:50])

[27, 0, 20, 0, 26, 5, 15, 0, 17, 0, 14, 6, 3, 5, 15, 3, 5, 25, 0, 20, 0, 15, 5, 20, 0, 17, 0, 14, 5, 27, 0, 20, 0, 26, 5, 15, 0, 17, 0, 14, 6, 3, 5, 15, 3, 5, 25, 13, 5, 3]


악보 데이터인 'text'를 앞에서 생성한 딕셔너리를 이용하여 숫자 데이터 'num_text' 변형

#### **2.2 train_X, train_Y 데이터 만들기**

train_X 데이터는 순서대로 아무거나 10개를 고른 후 train_Y 데이터는 그 다음 문자로 설정한다.

10개 정도면 다음에 나올 음을 충분히 예측할 수 있다고 판단

In [13]:
train_x = []
train_y = []

for i in range(0, len(num_text) - 10):
  train_x.append(num_text[i:i+10]) 
  train_y.append(num_text[i+10])

print(train_x[0:3])
print(train_y[0:3])

[[27, 0, 20, 0, 26, 5, 15, 0, 17, 0], [0, 20, 0, 26, 5, 15, 0, 17, 0, 14], [20, 0, 26, 5, 15, 0, 17, 0, 14, 6]]
[14, 6, 3]


In [14]:
print(np.array(train_x).shape)
print(np.array(train_y).shape)

(292012, 10)
(292012,)


총 292012개의 train data가 만들어졌다.

#### **2.3 원핫 인코딩**
모델에 이 정수들을 그대로 넣을 수도 있지만 원핫 인코딩을 한 후 넣을 수도 있다.

unique_text가 많으면 원핫 인코딩 더 부담이 될 수 있다.

In [15]:
train_X = tf.one_hot(train_x, 31)
train_Y = tf.one_hot(train_y, 31)

print(train_X[0])
print(train_Y[0])
print(train_X.shape)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0.]], shape=(

원 핫인코딩이 된 것을 확인할 수 있다.

tensorflow에서 제공하는 `tf.one_hot([unique 문자 수])` 함수를 이용하면 쉽게 원 핫 인코딩 형태로 데이터를 변형할 수 있다.

### **3. 모델 제작하기**

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.LSTM(100, input_shape=(10, 31)),
  tf.keras.layers.Dense(31, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

`tf.keras.laers.LSTM([노드 수], input_shape=([하나의 데이터 shape]))` lstm 레이어에는 활성화함수가 필요없다.

`tf.keras.layers.Dense(31, activation='softmax')` 마지막 출력에서는 unique text 31개 중 하나를 출력해야 하기 때문에 노트가 31개인 것을 확인할 수 있다.

데이터가 원핫인코딩이 안되어 있다면 loss 함수로 `sparse_categorical_crossentropy`를 넣어줘야한다.

### **4. 모델 학습시키기**

In [ ]:
model.fit(train_X, train_Y, batch_size=64, epochs=30, verbose=2)
model.save('/content/drive/MyDrive/강의/Coding Apple - 문과여도 상관없다 Tensorflow 딥러닝 AI 기초부터 실무까지/model1')

Epoch 1/30
4563/4563 - 33s - loss: 0.9212 - accuracy: 0.6980 - 33s/epoch - 7ms/step
Epoch 2/30
4563/4563 - 33s - loss: 0.9175 - accuracy: 0.6999 - 33s/epoch - 7ms/step
Epoch 3/30
4563/4563 - 33s - loss: 0.9131 - accuracy: 0.7011 - 33s/epoch - 7ms/step
Epoch 4/30
4563/4563 - 33s - loss: 0.9095 - accuracy: 0.7019 - 33s/epoch - 7ms/step
Epoch 5/30
4563/4563 - 33s - loss: 0.9057 - accuracy: 0.7032 - 33s/epoch - 7ms/step
Epoch 6/30
4563/4563 - 33s - loss: 0.9022 - accuracy: 0.7039 - 33s/epoch - 7ms/step
Epoch 7/30
4563/4563 - 33s - loss: 0.8986 - accuracy: 0.7061 - 33s/epoch - 7ms/step
Epoch 8/30
4563/4563 - 34s - loss: 0.8958 - accuracy: 0.7063 - 34s/epoch - 7ms/step
Epoch 9/30
4563/4563 - 34s - loss: 0.8924 - accuracy: 0.7083 - 34s/epoch - 8ms/step
Epoch 10/30
4563/4563 - 35s - loss: 0.8888 - accuracy: 0.7087 - 35s/epoch - 8ms/step
Epoch 11/30
4563/4563 - 33s - loss: 0.8864 - accuracy: 0.7098 - 33s/epoch - 7ms/step
Epoch 12/30
4563/4563 - 33s - loss: 0.8834 - accuracy: 0.7110 - 33s/epoch 

INFO:tensorflow:Assets written to: /content/drive/MyDrive/강의/Coding Apple - 문과여도 상관없다 Tensorflow 딥러닝 AI 기초부터 실무까지/model1/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/강의/Coding Apple - 문과여도 상관없다 Tensorflow 딥러닝 AI 기초부터 실무까지/model1/assets


`batch_size=[]`는 몇번의 학습 후 w값이 업데이트가 일어나게 하고 싶은지 설정해주는 파라미터다. 모든 데이터를 다 넣어준 후 마지막에 w 값을 업데이트하고 싶다면 안 넣어주면 된다.

LSTM은 학습하는데 시간이 좀 걸린다.

verbose=2는 출력을 좀 덜 해달라는 파라미터다. epoch 길게 학습할 경우 필수

### **5. 예측하기**

In [7]:
model = tf.keras.models.load_model("/content/drive/MyDrive/강의/Coding Apple - 문과여도 상관없다 Tensorflow 딥러닝 AI 기초부터 실무까지/model1")

예측은 깔끔하게 볼 수 있도록 다른 파이썬 파일에서 하기도 함. 이번엔 그냥 같은 파일에서 진행하겠다.

In [16]:
input = num_text[100:100+10]
input = tf.one_hot(input, 31)
input = tf.expand_dims(input, axis=0) # 2차원 데이터를 3차원 데이터로 만들어주기
print(input)

tf.Tensor(
[[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 1. 0. 0.

원핫인코딩을 넣어줘서 학습을 진행했기 때문에 값을 예측할 때도 그대로 원핫 인코딩 된 데이터를 넣어줘야한다.

처음 학습 데이터가 3차원이기 때문에 예측할 때 값도 3차원으로 넣어줘야한다.

In [17]:
predict = model.predict(input) 
print(predict[0])

print(np.argmax(predict[0])) # 확률로 나타나 있는 예측값을 최대 확률인 숫자를 출력하는 것

[1.6269380e-03 1.5416192e-06 3.7319587e-06 5.4875791e-01 1.0360702e-05
 1.3036336e-03 8.5865475e-02 6.9423164e-05 2.7407054e-10 6.7290884e-09
 6.1497039e-06 2.1777322e-07 6.4393894e-12 3.2471956e-04 2.9852629e-01
 2.4004215e-03 5.5104724e-06 1.6854152e-03 7.6448334e-05 6.6514928e-03
 8.1476057e-05 8.8172185e-04 6.4546418e-07 3.3429898e-02 8.6504304e-05
 3.2148554e-03 4.9754800e-03 1.3213252e-03 8.2831876e-03 4.9149163e-05
 3.6007180e-04]
3


31개의 확률을 출력한다.

`argmax()`를 이용하여 가장 최대값을 가지는 인덱스를 출력할 수 있다.

예측한 값이 실제 학습한 데이터와 일치한 것을 볼 수 있다.

### **6. 노래 작곡해보기**

In [18]:
music = []

for i in range(500):
  predict2 = model.predict(input)
  predict2 = np.argmax(predict2[0]) # 예측하기 np.random.choice

  music.append(predict2) # 예측한 값 music 리스트에 추가

  input2 = input.numpy()[0][1:] # 입력값에서 첫번째 값 삭제

  
  one_hot_num = tf.one_hot(predict2, 31) # 예측값 원핫인코딩
  input = np.vstack([input2, one_hot_num.numpy()]) # 첫번째 값을 삭제한 기존의 입력값 뒤에 새로 예측한 값 추가
  input = tf.expand_dims(input, axis=0) # 3차원으로 변형

print(music)

[3, 5, 23, 3, 7, 23, 3, 7, 0, 18, 0, 29, 3, 7, 28, 3, 7, 27, 3, 7, 28, 3, 7, 0, 17, 0, 26, 3, 5, 28, 3, 5, 0, 14, 0, 27, 3, 5, 25, 3, 5, 14, 3, 5, 20, 3, 5, 20, 3, 5, 14, 3, 5, 0, 20, 0, 15, 20, 20, 13, 28, 3, 5, 27, 3, 5, 0, 17, 0, 26, 3, 5, 25, 3, 5, 26, 3, 5, 27, 3, 5, 28, 3, 5, 29, 3, 5, 0, 14, 0, 23, 3, 5, 27, 3, 5, 25, 3, 5, 27, 3, 5, 25, 3, 5, 0, 17, 0, 26, 3, 5, 27, 3, 5, 28, 3, 5, 29, 3, 5, 0, 14, 0, 23, 3, 5, 27, 3, 5, 25, 3, 5, 27, 3, 5, 25, 3, 5, 0, 17, 0, 26, 3, 5, 27, 3, 5, 28, 3, 5, 29, 3, 5, 0, 14, 0, 23, 3, 5, 27, 3, 5, 25, 3, 5, 27, 3, 5, 25, 3, 5, 0, 17, 0, 26, 3, 5, 27, 3, 5, 28, 3, 5, 29, 3, 5, 0, 14, 0, 23, 3, 5, 27, 3, 5, 25, 3, 5, 27, 3, 5, 25, 3, 5, 0, 17, 0, 26, 3, 5, 27, 3, 5, 28, 3, 5, 29, 3, 5, 0, 14, 0, 23, 3, 5, 27, 3, 5, 25, 3, 5, 27, 3, 5, 25, 3, 5, 0, 17, 0, 26, 3, 5, 27, 3, 5, 28, 3, 5, 29, 3, 5, 0, 14, 0, 23, 3, 5, 27, 3, 5, 25, 3, 5, 27, 3, 5, 25, 3, 5, 0, 17, 0, 26, 3, 5, 27, 3, 5, 28, 3, 5, 29, 3, 5, 0, 14, 0, 23, 3, 5, 27, 3, 5, 25, 3, 5, 27, 3, 

In [19]:
music_text = []

for i in music:
  music_text.append(num_to_text[i])

print(''.join(music_text))

/2a/4a/4"E"g/4f/4e/4f/4"D"d/2f/2"A"e/2c/2A/2G/2G/2A/2"G"BGG:f/2e/2"D"d/2c/2d/2e/2f/2g/2"A"a/2e/2c/2e/2c/2"D"d/2e/2f/2g/2"A"a/2e/2c/2e/2c/2"D"d/2e/2f/2g/2"A"a/2e/2c/2e/2c/2"D"d/2e/2f/2g/2"A"a/2e/2c/2e/2c/2"D"d/2e/2f/2g/2"A"a/2e/2c/2e/2c/2"D"d/2e/2f/2g/2"A"a/2e/2c/2e/2c/2"D"d/2e/2f/2g/2"A"a/2e/2c/2e/2c/2"D"d/2e/2f/2g/2"A"a/2e/2c/2e/2c/2"D"d/2e/2f/2g/2"A"a/2e/2c/2e/2c/2"D"d/2e/2f/2g/2"A"a/2e/2c/2e/2c/2"D"d/2e/2f/2g/2"A"a/2e/2c/2e/2c/2"D"d/2e/2f/2g/2"A"a/2e/2c/2e/2c/2"D"d/2e/2f/2g/2"A"a/2e/2c/2e/2c/


**해결해야할 점**

- 계속 같은 구간을 반복한다.
- 예측한 값 abc notation이 오류가 일어나는 경우가 있다. 